In [29]:
import pandas as pd

import logging

logging.basicConfig(filename='00-runall.log' ,format='%(asctime)s | %(levelname)s: %(message)s', level=20)
logging.info('I start to run your Notebook ! -- Good Luck :-) ')


In [49]:
# declare some constants
path_data_source = '../../data source/'
save_folder = '../../data source/_anonymized/'

In [31]:
w = pd.read_csv(path_data_source+'MOCK_DATA.csv')

In [32]:
w

,caviar
0,Balearica pavonina
1,Gekko gecko
2,Lepilemur rufescens
3,Echimys chrysurus
4,Melophus lathami
...,...
995,Felis libyca
996,Ara ararauna
997,Centrocercus urophasianus
998,Choriotis kori


In [33]:
# load the file from source repository

# NAME = fund name
ds1 = pd.read_csv(path_data_source+'funds_scope.csv', delimiter=';')

# Asset Name & Produt Strategy
ds_flows_a = pd.read_csv(path_data_source+'monthly_flows_2010-2013.csv', delimiter=';', nrows=None, thousands=' ')
ds_flows_b = pd.read_csv(path_data_source+'monthly_flows_2014.csv', delimiter=';', nrows=None, thousands=' ')
ds_flows_c = pd.read_csv(path_data_source+'monthly_flows_2015-2016.csv', delimiter=';', nrows=None, thousands=' ')
ds_flows_d = pd.read_csv(path_data_source+'monthly_flows_2017-2018.csv', delimiter=';', nrows=None, thousands=' ')
ds_flows_e = pd.read_csv(path_data_source+'monthly_flows_2019-2021.csv', delimiter=';', nrows=None, thousands=' ')

ds_flows = pd.concat([ds_flows_a, ds_flows_b, ds_flows_c, ds_flows_d, ds_flows_e], ignore_index=True)

# UMBREALLA, FUND NAME & MANAGER
ds4 = pd.read_csv(path_data_source+'funds-managers.csv', delimiter=';')

# FUND
ds9 = pd.read_csv(path_data_source+'monthly_documents_downloaded.csv', delimiter=';')

# FUND_NAME
ds10 = pd.read_csv(path_data_source+'fund_fees_adm_mgt.csv', delimiter=';', decimal=',')

# RFP
# create a list of files recevied by the RFP Team
rfp_files = [path_data_source+'rfp.csv',
             path_data_source+'rfp2013.csv',
             path_data_source+'rfp2016.csv',
             path_data_source+'rfp2015.csv']

# use a few columns only
usecols = ['Status','RFP ID','Product Type','Creation Date',
        'Product']
ds11 = pd.DataFrame()
for file in rfp_files:
    ds11 = pd.concat( [ds11, pd.read_csv( file, delimiter=';', usecols=usecols, decimal=',',
                                         error_bad_lines=False, warn_bad_lines=False )], axis=0, sort=True )
    
# I have a special file with data for RFP from 2017 to 2021. I will upload it separatly and make some transormation (columns header) to match other files
usecols = ['Status','Production Deadline','Product Name']
#names = ['Status','Creation Date','Product']
rfp1 = pd.read_csv(path_data_source+ 'rfp2017-2021-cleaned.csv', usecols=usecols,  delimiter=';',
                   error_bad_lines=False, warn_bad_lines=False )
rfp1.rename({'Production Deadline':'Creation Date','Product Name':'Product'}, axis=1, inplace=True)

ds11 = pd.concat([ds11,rfp1], axis=0, sort=True)

# drop row from rfp with no product or date
ds11.dropna(axis=0,subset=['Product','Creation Date'], inplace=True)

# event marketing impacts
ds12 = pd.read_csv(path_data_source+'export-events-SF.csv', delimiter=',', nrows=None)

# capabilities (Strategic axes)
ds13 = pd.read_csv(path_data_source + 'strategy_capabilities.csv', 
                   delimiter=';',error_bad_lines=False, warn_bad_lines=False ) 
ds13.rename({'Sub Fund Mispf Id':'SRC_UID'}, axis=1, inplace=True)

In [34]:

ds13.head()

,SRC_UID,Fund name,Compartment name,Strategy,Asset Class,Sub Asset Class,Strategy Cluster,Strategy Capability,Capability IS Strategic
0,198188413318323,Pictet TR,Agora,Agora,Alternatives,Long Short,Equity L/S,Absolute Return,Yes
1,197463014939188,Pictet TR,Diversified Alpha,Alphanatics,Alternatives,Multistrategy,Multistrategy L/S,Absolute Return,Yes
2,219007141108041,Pictet TR,Aquila,Aquila,Alternatives,Long Short,Equity L/S,Absolute Return,Yes
3,219007141108045,Pictet TR,Atlas Titan,Atlas,Alternatives,Long Short,Equity L/S,Absolute Return,Yes
4,208480774306900,Pictet TR,Atlas,Atlas,Alternatives,Long Short,Equity L/S,Absolute Return,Yes


In [35]:
ds1.shape

(195, 7)

In [36]:
ds1 = ds1.merge(ds13.loc[:,['SRC_UID','Strategy Capability','Capability IS Strategic']], how='inner',
               left_on = 'SRC_UID', right_on='SRC_UID')
ds1.shape

(92, 9)

In [37]:
ds1.sample(8)

,MULTI_MANAGER_STRUCTURE,NAME,SOFTCLOSING,SRC_UID,RISK_LEVEL,DATE_SHARE,CNT_SHARE,Strategy Capability,Capability IS Strategic
69,1,Short-Term Money Market EUR,0.0,206670690,Low risk,09.01.1998,10,Money Market,No
44,1,Japan Index,0.0,1489764652,Moderate / high risk,06.06.2002,21,Indexed Equities,No
8,1,Clean Energy,0.0,272675524722,High risk,14.05.2007,26,Thematic,Yes
28,0,Enhanced Liquidity EUR,0.0,108202450315,Low risk,23.06.2005,5,Money Market,No
79,0,Sovereign Short-Term Money Market EUR,0.0,755274358166,Low risk,26.03.2008,5,Money Market,No
56,1,Piclife,0.0,206670760,Moderate risk,18.08.1993,7,Multi Asset,Yes
12,0,Diversified Alpha,1.0,197463014939188,Moderate risk,01.05.2014,18,Absolute Return,Yes
37,0,Global Equities,0.0,52079200642,Moderate / high risk,05.04.2004,6,Indexed Equities,No


In [38]:
f = ds1['Strategy Capability'].isnull()
ds1[f]

,MULTI_MANAGER_STRUCTURE,NAME,SOFTCLOSING,SRC_UID,RISK_LEVEL,DATE_SHARE,CNT_SHARE,Strategy Capability,Capability IS Strategic


In [39]:
ds11.isnull().sum()

Creation Date       0
Product             0
Product Type     3821
RFP ID           2628
Status              0
dtype: int64

## merge all columns which contain text

In [40]:
s1 = ds1.NAME
s2a = ds_flows['Asset Name']
s2b = ds_flows['Product Strategy']
s4a = ds4.UMBRELLA
s4b = ds4['FUND NAME']
s4c = ds4.MANAGER
s9 = ds9.FUND
s10 = ds10.FUND_NAME
s11 = ds11.Product

In [41]:
# merge all series
all_words = pd.concat([s1, s2a, s2b, s4a, s4b, s4c, s9, s10, s11], ignore_index=True)

all_words = pd.DataFrame(all_words)

all_words.drop_duplicates(inplace=True)

all_words[:1]

,0
0,Agora


In [42]:
all_words.reset_index(inplace=True)

In [43]:

all_caviar = w.copy()

In [44]:
anonym=all_words.merge(all_caviar, left_index=True, right_index=True)
anonym.drop('index', axis=1, inplace=True)
anonym=anonym.set_index(0)
anonym=anonym.to_dict()
anonym=anonym['caviar']

In [45]:
first2pairs = {k: anonym[k] for k in list(anonym)[:2]}
first2pairs

{'Agora': 'Balearica pavonina', 'Asian Equities Ex Japan': 'Gekko gecko'}

In [46]:
# change text data and save file in _anonymized folder

def anonymize_col(ds,col):
    ds_anonym = ds.copy()
    for idx, r in ds_anonym.iterrows():
        col_value=r[col]
        new_value=anonym[col_value]
        ds_anonym.loc[idx,col]=new_value
    
    return ds_anonym

In [47]:
ds1_a = anonymize_col(ds1,'NAME')
ds_flows_a = anonymize_col(ds_flows,'Asset Name')
ds_flows_a = anonymize_col(ds_flows_a,'Product Strategy')
ds4_a = anonymize_col(ds4,'UMBRELLA')
ds4_a = anonymize_col(ds4_a,'FUND NAME')
ds4_a = anonymize_col(ds4_a,'MANAGER')
ds9_a = anonymize_col(ds9,'FUND')
ds10_a = anonymize_col(ds10,'FUND_NAME')
ds11_a = anonymize_col(ds11,'Product')

In [50]:
# save anonymized files
ds1_a.to_csv(save_folder + 'funds_scope.csv')
ds_flows_a.to_csv(save_folder + 'ds_flows.csv')
ds4_a.to_csv(save_folder + 'funds-managers.csv')
ds9_a.to_csv(save_folder + 'monthly_documents_downloaded.csv')
ds10_a.to_csv(save_folder + 'fund_fees_adm_mgt.csv')
ds11_a.to_csv(save_folder + 'monthly_rfp.csv')

In [51]:
ds12 = ds12.drop_duplicates().dropna().reindex()
ds12.drop('Campaign Name', axis=1)

,Total Attended,Total Invited,Total Accepted,End Date
0,0,6199,215,29.04.2021
6199,0,1,1,22.04.2021
6200,0,5,5,15.04.2021
6205,0,3,3,12.04.2021
6208,0,269,269,09.04.2021
...,...,...,...,...
1069422,67,78,0,12.01.2015
1069500,3,3,0,15.05.2013
1069511,16,16,0,26.03.2012
1069529,9,9,0,01.04.2011


### Event Impact:
I will calculate a rating of the event by looking the 3 values
- Invited
- Accepted
- Attended

For event with acceptation process, the ratio between attendees and accepted will rank the event.
When the acceptation process was not in the process, the ratio to qualify the rank will be a ratio between the invited and attendee.
When invited and accepted numbers are not available, the event will have a neutral rank.

For month (year + month) when we do not have information, the neutral rank will be assigned assuming that we are always (at least every month) proposing client intercation (Event, Webcast, Conference).

In [52]:
def rank_event( row ):
    # initialize rank to -1
    rank = -1
    # rank the event and return the value
    # if accepted is > 0, calculate the rank with accepted Vs. attended
    if row['Total Attended'] == 0:
        if row['Total Invited']>0:
            rank = row['Total Accepted'] / row['Total Invited']
    if row['Total Accepted']>0:
        if row['Total Attended'] > 0: 
            rank = row['Total Attended'] / row['Total Accepted']
        else:
            rank = row['Total Accepted'] / row['Total Invited']    
    else:
        if row['Total Invited']>0:
            rank = row['Total Attended'] / row['Total Invited']
            
    return abs(rank)


In [53]:
ds12['EVENT_IMPACT'] = ds12.apply(rank_event, axis=1)
ds12['End Date'] = pd.to_datetime(ds12['End Date'])

In [54]:
ds12.reset_index(inplace=True)

In [55]:
ds12=ds12[['End Date','EVENT_IMPACT']].sort_values('End Date')
ds12.reset_index(inplace=True, drop=True)
ds12

,End Date,EVENT_IMPACT
0,2010-04-23,1.000000
1,2011-01-04,1.000000
2,2012-03-26,1.000000
3,2013-05-15,1.000000
4,2015-01-03,0.392593
...,...,...
1509,2021-11-03,1.770777
1510,2021-11-03,0.000000
1511,2021-11-03,0.090909
1512,2021-12-01,0.027982


In [56]:
ds12.to_csv(save_folder + 'events_impacts.csv')

In [57]:
logging.info('I did my job ! -- Good Luck for next Notebook :-) ')